In [92]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
import os

spark = SparkSession.builder.getOrCreate()

In [28]:
read_path = "/home/genicleito/Documents/GitHub/mate33/data/01_SINASC.csv"

In [29]:
sinasc = spark.read.csv(read_path, header = True)

In [30]:
sinasc.count()

1503692

### Step 1

In [68]:
ranges = []
for c in sinasc.columns:
    ranges.append(sinasc.describe(c))

In [211]:
def triangularFuzzyRegionsPoints(describes, nRegions):
    dic = {}
    fuzzyRegionsPoints = []
    for describe in describes:
        column_name = describe.columns[1]
        range_c = [float(x[1]) for x in describe.collect()][-2:]
        dic[describe.columns[1]] = range_c
        
        intervalLength = abs(range_c[1] - range_c[0])
        
        if nRegions > 1:
            distInterPoints = intervalLength / (nRegions - 1)
        else:
            trian_func_degrees = [0, 1, 0]
            distInterPoints = intervalLength / 2
            pointsFuzzyRegions = [str(x) for x in np.arange(range_c[0], range_c[1] + distInterPoints, distInterPoints)]
            fuzzyRegionsPoints.append(spark.createDataFrame([{"values_" + column_name:pointsFuzzyRegions[i], "degree_" + column_name:trian_func_degrees[i]} for i in range(len(pointsFuzzyRegions))]))
            continue
        
        pointsFuzzyRegions = [str(x) for x in np.arange(range_c[0], range_c[1] + distInterPoints, distInterPoints)]
        
        allPoints = []
        allDegrees = []
        for i in range(len(pointsFuzzyRegions)):
            if i == 0:
                allPoints = [str(x) for x in np.repeat(pointsFuzzyRegions[i], 2)] + [pointsFuzzyRegions[i + 1]]
                allDegrees = [1, 1, 0]
            elif  i == len(pointsFuzzyRegions) - 1:
                allPoints += [pointsFuzzyRegions[i - 1]] + [str(x) for x in np.repeat(pointsFuzzyRegions[i], 2)]
                allDegrees += [0, 1, 1]
            else:
                allPoints += pointsFuzzyRegions[(i - 1):(i + 2)]
                allDegrees += [0, 1, 0]
        fuzzyRegionsPoints.append(spark.createDataFrame([{"values_" + column_name:allPoints[i], "degree_" + column_name:allDegrees[i]} for i in range(len(allPoints))]))
    return fuzzyRegionsPoints
#     return dic
# triangularFuzzyRegionsPointsUDF = F.udf(triangularFuzzyRegionsPoints, StringType())

In [218]:
regionsPoints = triangularFuzzyRegionsPoints(ranges, 5)

In [220]:
for rp in regionsPoints:
    rp.show()

+---------------+---------------+
|degree_idademae|values_idademae|
+---------------+---------------+
|              1|           12.0|
|              1|           12.0|
|              0|           25.0|
|              0|           12.0|
|              1|           25.0|
|              0|           38.0|
|              0|           25.0|
|              1|           38.0|
|              0|           51.0|
|              0|           38.0|
|              1|           51.0|
|              0|           64.0|
|              0|           51.0|
|              1|           64.0|
|              1|           64.0|
+---------------+---------------+

+--------------+--------------+
|degree_locnasc|values_locnasc|
+--------------+--------------+
|             1|           1.0|
|             1|           1.0|
|             0|           2.0|
|             0|           1.0|
|             1|           2.0|
|             0|           3.0|
|             0|           2.0|
|             1|           3.0|
|

+-----------------+-----------------+
|degree_qtdgestant|values_qtdgestant|
+-----------------+-----------------+
|                1|              0.0|
|                1|              0.0|
|                0|              2.5|
|                0|              0.0|
|                1|              2.5|
|                0|              5.0|
|                0|              2.5|
|                1|              5.0|
|                0|              7.5|
|                0|              5.0|
|                1|              7.5|
|                0|             10.0|
|                0|              7.5|
|                1|             10.0|
|                1|             10.0|
+-----------------+-----------------+

+----------------+----------------+
|degree_mesprenat|values_mesprenat|
+----------------+----------------+
|               1|             1.0|
|               1|             1.0|
|               0|             3.0|
|               0|             1.0|
|               1|       

### Step 2

In [216]:
#DOING...